<a href="https://colab.research.google.com/github/deataxe/pw/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22roop_face_replacement_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 1) Проверяем видеокарту/Check graphics card
!nvidia-smi

Sat Apr 20 13:27:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#@title 2) Подготовка
!git clone https://github.com/Obnovlator3000/roop
%cd /content/roop
!pip install onnxruntime-gpu && pip install -r requirements.txt
!wget https://huggingface.co/thebiglaskowski/inswapper_128.onnx/resolve/main/inswapper_128.onnx

Cloning into 'roop'...
remote: Enumerating objects: 1262, done.
remote: Total 1262 (delta 0), reused 0 (delta 0), pack-reused 1262
Receiving objects: 100% (1262/1262), 90.63 MiB | 23.19 MiB/s, done.
Resolving deltas: 100% (773/773), done.
/content/roop
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring onnxruntime: markers 'sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring tensorflow: markers 'sys_platform == "darwin"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.6 MB/s eta 0:00:00
     ━━━━━━

--2024-04-20 13:32:45--  https://huggingface.co/thebiglaskowski/inswapper_128.onnx/resolve/main/inswapper_128.onnx
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.88, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/69/55/6955e7aef1f96409a7afeb15ed35cbcf5439916338eb5622c66f01d89d29944d/e4a3f08c753cb72d04e10aa0f7dbe3deebbf39567d4ead6dce08e98aa49e16af?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27inswapper_128.onnx%3B+filename%3D%22inswapper_128.onnx%22%3B&Expires=1713879165&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzg3OTE2NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82OS81NS82OTU1ZTdhZWYxZjk2NDA5YTdhZmViMTVlZDM1Y2JjZjU0Mzk5MTYzMzhlYjU2MjJjNjZmMDFkODlkMjk5NDRkL2U0YTNmMDhjNzUzY2I3MmQwNGUxMGFhMGY3ZGJlM2RlZWJiZjM5NTY3ZDRlYWQ2ZGNl

In [4]:
#@title 3) Загружаем фото с лицом/Upload photo with face
%cd /content/roop
!mkdir files
%cd /content/roop/files
from google.colab import files
uploaded = files.upload()
for i in uploaded.keys():
  face_raw = "/content/roop/files/" + i
  break
!mv "$face_raw" /content/roop/files/face.jpg

/content/roop
/content/roop/files


Saving unnamed.jpg to unnamed.jpg


In [5]:
#@title 4) Загружаем видео/Upload video
%cd /content/roop/files
from google.colab import files
uploaded = files.upload()
for i in uploaded.keys():
  video_raw = "/content/roop/files/" + i
  break
!mv "$video_raw" /content/roop/files/video.mp4

/content/roop/files


Saving IMG_1620.MP4 to IMG_1620.MP4


In [ ]:
#@title 5) Заменяем лицо/Face swap
#@markdown Заменять все лица:
swap_all_faces  = True #@param{type:"boolean"}
if swap_all_faces:
  all_faces = "--all-faces"
else:
  all_faces = ""
%cd /content/roop/
!touch /content/output.mp4
c = f"python run.py -f /content/roop/files/face.jpg -t /content/roop/files/video.mp4 -o /content/output.mp4 --gpu-vendor nvidia --gpu-cores 3 {all_faces}"
get_ipython().system(c)
from google.colab import files
files.download('/content/output.mp4')

/content/roop
2024-04-20 13:42:24.786618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 13:42:26.818204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-20 13:42:33.955195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-20 13:42:34.519379: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would 